# **NLP com Flair**
Este tutorial é baseado no livro "Natural Language Processing with Flair".

# 1 - Sentenças, Tokens e Embeddings

## Sentenças

Primeiro vamos explorar um dos objetos mais fundamentais do Flair, a `Sentence`. 

In [1]:
from flair.data import Sentence

sentence = Sentence('Um exemplo de sentença.')
print(sentence, len(sentence))

/home/davibarreira/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sentence: "Um exemplo de sentença ." 5


Note que a nossa sentença "Um exemplo de sentença", o Flair identificou 5 tokens.
Uma token é uma "unidade de texto". Uma unidade de texto comum de se usar
é "palavra". Ou seja, uma sentença é composta de palavras individuais. Porém, além de palavras, podemos ter coisas como
pontuação ou nome próprios. Por exemplo, "João Miguel foi à praia.".  Podemos identificar "João Miguel" como uma token só.
Isso faz mais sentido na hora de analisar esse texto, pois "João Miguel" é na verdade o nome de uma só pessoa, e não dois indivíduos.
De forma similar, na sentença "A Universidade Federal do Ceará é uma boa universidade", temos a token "Universidade Federal do Ceará"
que simboliza uma só entidade.

A maneira de identificar essas "unidades de texto", ou, tokens, varia. Você pode querer simplesmente quebrar por palavras, ou usar
algum outro método. Os modelos que fazem esse tipo de trabalho são os chamado tokenizadores (tokenizer).

No Flair, quando não passamos um tokenizador, ele por default utiliza um modelo próprio chamado `SegTokTokenizer`.
Esse modelo tokeniza utilizando alguns critérios razoáveis, como pontuação, e espaços em branco.

Vamos agora trocar o tokenizador pra mostrar como isso impacta na identificação das nossas tokens.

In [2]:
from flair.data import Sentence
from flair.tokenization import SpaceTokenizer
from flair.tokenization import SpacyTokenizer
tokenizer = SpaceTokenizer()
s = Sentence('Um exemplo de sentença.', use_tokenizer=tokenizer)
# getting the string representation using magic method __str__ 
print(s, len(s))

Sentence: "Um exemplo de sentença." 4


Agora, invés de 5, obtivemos 4, pois no modelo `SpaceTokenizer`, somente espaços " " são usados pra tokenizar. 

O objeto `Sentence` é composto de tokens, que podem ser acessada facilmente.
Ele pode ser acessado como uma lista, ou usando o método `get_token`. Note
que o `get_token` começa a indexação por 1 invés de 0 (padrão de python).

In [3]:
s.get_token(1).add_label('manual-pos', 'DT')
print(s)


print(s[0])
for token in s:
    print(token)
    
len(s), len(s.to_original_text())

Sentence: "Um exemplo de sentença." → ["Um"/DT]
Token[0]: "Um" → DT (1.0)
Token[0]: "Um" → DT (1.0)
Token[1]: "exemplo"
Token[2]: "de"
Token[3]: "sentença."


(4, 23)

In [4]:
from flair.tokenization import SpacyTokenizer
import spacy

In [5]:
model = spacy.load("pt_core_news_lg")
tokenizer = SpacyTokenizer(model)

s = Sentence('Moro na cidade de São Paulo.', use_tokenizer=tokenizer)
print(s, len(s))
for token in s:
    print(token)

Sentence: "Moro na cidade de São Paulo ." 7
Token[0]: "Moro"
Token[1]: "na"
Token[2]: "cidade"
Token[3]: "de"
Token[4]: "São"
Token[5]: "Paulo"
Token[6]: "."


## Tokenizer Personalizado

In [7]:
from flair.data import Token
from flair.tokenization import TokenizerWrapper

def char_splitter(string):
    return list(string)
char_tokenizer = TokenizerWrapper(char_splitter)

In [8]:
text = "Teste texto."
sentence = Sentence(text, use_tokenizer=char_tokenizer)
for token in sentence:
    print(token)

Token[0]: "T"
Token[1]: "e"
Token[2]: "s"
Token[3]: "t"
Token[4]: "e"
Token[5]: " "
Token[6]: "t"
Token[7]: "e"
Token[8]: "x"
Token[9]: "t"
Token[10]: "o"
Token[11]: "."


## Corpus

Um corpus é uma coleção de textos. No Flair existem alguns corpus padrões que podemos importar.
Esses corpus já vem dividos em train, test e dev. Nesse caso, o dev é o que normalmente chamamos de validation set.

In [9]:
from flair import datasets

corpus = datasets.UD_PORTUGUESE()
print(corpus)

2022-10-18 10:57:19,566 Reading data from /home/davibarreira/.flair/datasets/ud_portuguese
2022-10-18 10:57:19,570 Train: /home/davibarreira/.flair/datasets/ud_portuguese/pt_bosque-ud-train.conllu
2022-10-18 10:57:19,571 Dev: /home/davibarreira/.flair/datasets/ud_portuguese/pt_bosque-ud-dev.conllu
2022-10-18 10:57:19,573 Test: /home/davibarreira/.flair/datasets/ud_portuguese/pt_bosque-ud-test.conllu
Corpus: 7018 train + 1172 dev + 1167 test sentences


In [10]:
train_dataset = corpus.train

## Embeddings

Vamos recapitular a estrutura hierárquica que temos em NLP. A maior unidade é o corpus, que é uma coleção de textos. A segunda unidade é uma "sentença". Aqui,
uma sentença não significa uma frase, mas simplesmente o texto que estamos focando pra realizar algum tipo de análise.
Por exemplo, podemos ter um corpus de reviews de filmes. Cada review seria tratada como uma "sentença", que compõe o corpus.
A terceira unidade é a token, que, como já falamos, é análogo à ideia de palavras.

Com isso definido, o passo seguinte é entender como representar essas tokens no computador. A ideia é que a token (e.g. "carro")
deve ser representada de forma que possamos manipulá-la de forma inteligente. Por exemplo, seria interessante que de alguma
forma conseguissemos dizer que a token "carro" é parecida/próxima da token "veículo". A representação de uma token é o que chamamos
de embedding. No geral, a ideia envolve transformar um texto é um objeto matemático com propriedades interessantes (e.g. um vetor).

Dito isso, existem vários métodos diferentes para fazer um embedding. Vamos começar com alguns dos clássicos.
A lista de modelos disponibilizados pelo Flair pode ser achados [aqui](https://github.com/flairNLP/flair/blob/master/resources/docs/embeddings/CLASSIC_WORD_EMBEDDINGS.md).

Um tipo comum de embedding é o one-hot-encoding, onde simplesmente pegamos um texto e fazemos cada coluna como uma palavra, sendo 0 ou 1 caso
aquela token seja essa palavra específica.
O problema desse tipo de representação é se tivermos uma palavra nova, ela não será contemplada. Além disso, teremos colunas demais, e não será
possível fazer o tipo de manipulação que comentamos, onde queremos que "carro" se pareça com "veículo".

Para possibilitar esse tipo de "matemática das tokens", precisamos de um modelo mais complexo, e que de alguma forma tenha sido
treinado em textos daquela língua (e.g. português), de forma a entender que "carro" e "veículo" ocorrem em contextos parecidos,
e, portanto, são palavras parecidas.

In [11]:
from flair.embeddings import WordEmbeddings

fasttext = WordEmbeddings('pt')

O código acima baixou o modelo de embedding chamado "FastText" treinado em textos em português. Se você está usando Linux,
o Flair criou uma pasta `/home/./flair` onde salvou os embeddings. Assim, ele não precisa baixar novamente toda vida que for usado.

In [12]:
sentenca = Sentence("Meu carro é azul. Ele é o veículo que uso para viajar.")
fasttext.embed(sentenca)

[Sentence: "Meu carro é azul . Ele é o veículo que uso para viajar ."]

O que aconteceu? O nosso modelo `fasttext` criou uma representação vetorial de cada token da nossa senteça. 
Essas representações ficam gravadas dentro do objeto do tipo `Sentence`, ou seja, dentro da variável `sentenca`.
Para acessar a representação vetorial, basta usar `sentenca[0].embedding`. Isso nos retornará
um objeto do tipo torch.Tensor do pacote Pytorch... Para transformar em um "vetor", podemos usar
`sentenca[0].embedding.tolist()`.

In [13]:
print(sentenca[1])
sentenca[1].embedding

Token[1]: "carro"


tensor([ 1.2702e-02, -1.9549e-01,  2.9307e-02, -1.3602e-01,  1.2380e-01,
         5.3440e-03,  2.0641e-01, -4.3588e-02, -1.9815e-01,  2.0293e-01,
         4.1532e-02,  4.1676e-01,  1.3775e-01,  4.7727e-01,  1.0198e-01,
        -1.3813e-01,  4.0064e-02, -7.5214e-02, -1.5032e-01, -6.4728e-03,
        -1.7995e-01, -5.2995e-01,  2.0694e-01, -3.2790e-01,  2.9773e-01,
        -1.0036e-02, -4.4665e-01, -3.4481e-01,  3.7361e-01, -2.5838e-01,
         4.4260e-02, -7.0744e-02, -1.5724e-01,  2.6081e-01, -3.1650e-01,
        -5.1831e-01,  3.2216e-01,  1.1580e-01, -2.2487e-01, -4.5416e-02,
         1.9262e-01,  4.2113e-01,  1.1376e-02,  2.2154e-01,  7.5462e-02,
         7.8880e-02,  1.3207e-01,  2.1083e-01, -1.7563e-01,  8.2525e-02,
        -2.7856e-02, -4.1808e-02, -1.6477e-01, -2.1031e-01, -2.3907e-01,
        -1.4270e-01, -1.2981e-01,  2.0888e-01, -3.4326e-02, -3.0938e-01,
         5.1215e-02,  3.1539e-01,  3.8521e-02, -1.4974e-01, -6.1015e-02,
        -3.9620e-01, -4.0661e-01, -3.0586e-01, -9.3

Vamos ver agora se as tokens "carro" e "veículo" tem uma representação parecida. Ou seja, se seus embeddings são similares.
Para isso, precisamos de alguma medida que meça a similaridade entre dois vetores.
Existem diversas forma de fazer isso. Uma das mais comuns é a similaridade de cossenos. Ela consiste simplesmente em medir o
ângulo entre os vetores, não se importando com sua magnitude. Dois vetores parecidos vão ter um ângulo entre eles de quase zero.
Podemos usar o produto interno para facilmente calcular o cosseno entre os vetores.
Se o cosseno entre os vetores der próximo à 1, eles são similares. Se der próximo à 0 ele são distintos.
Se der próximos à -1 eles tem significado oposto (e.g. "bom" e "mau").

A fórmula que vamos usar é simples:
$$
cos(\theta) = \frac{\langle \mathbf{u},\mathbf{v} \rangle}{||\mathbf{u}|| \ || \mathbf{v} ||}
$$

In [14]:
import numpy as np

def similirity_cos(word1, word2):
    s1 = Sentence(word1)
    s2 = Sentence(word2)
    fasttext.embed(s1)
    fasttext.embed(s2)
    inner_uv = np.inner(s1[0].embedding.tolist(),s2[0].embedding.tolist())
    norm_uv = np.linalg.norm(s1[0].embedding.tolist())*np.linalg.norm(s2[0].embedding.tolist())
    return inner_uv / norm_uv

carro = 'carro'
veiculo = 'veículo'
porta = 'porta'
bom = 'bom'
mau = 'mau'
otimo = 'ótimo'
pessimo = 'péssimo'

print(similirity_cos(carro, veiculo))
print(similirity_cos(bom, mau))
print(similirity_cos(carro, otimo))
print(similirity_cos(porta, otimo))

0.6452633282295562
0.5173644488701212
0.2285137382744788
0.12916372897649708


Implementamos a similaridade de cossenos. Porém, essa função já existe no scikit-learn. Vamos importar e checar com
nossa implementação.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

def sk_similirity_cos(word1, word2):
    s1 = Sentence(word1)
    s2 = Sentence(word2)
    fasttext.embed(s1)
    fasttext.embed(s2)
    return cosine_similarity([s1[0].embedding.tolist()], [s2[0].embedding.tolist()])

sk_similirity_cos(carro, veiculo)

array([[0.64526333]])

Caso queira remover um embedding já aplicado, basta usar o método `clear_embeddings()`. 

In [46]:
sentenca.clear_embeddings()

### Recuperando palavras de embeddings

E se quisermos o caminho inverso? Ou seja, a partir de um embedding chegar em uma palavra?
Isso pode ser feito, mas precisamos de um dicionário de palavras. Esse dicionário de palavras será representado
por uma nuvem de pontos no espaço vetorial. Assim, para um vetor qualquer, podemos
buscar qual outro vetor no nosso vocabulário que mais se aproxima dele.

Vamos criar uma função que faz o embedding de todo o vocabulário e retorna esse vocabulário. Note que
essa função vai retornar um objeto do tipo `Sentence` com o atributo de `embedding` já preenchido.

In [16]:
def get_embedded_pt_vocab(embedding):
    dataset = datasets.UD_PORTUGUESE()
    vocab_list = dataset.make_vocab_dictionary().get_items()
    vocab = Sentence(' '.join(vocab_list))
    embedding.embed(vocab)
    return vocab

Vamos testar a função olhando o resultado de uma das palavras (tokens) do nosso vocabulário. 

In [17]:
vocab = get_embedded_pt_vocab(fasttext) 
print(vocab[6], vocab[6].embedding)

2022-10-18 10:57:38,637 Reading data from /home/davibarreira/.flair/datasets/ud_portuguese
2022-10-18 10:57:38,637 Train: /home/davibarreira/.flair/datasets/ud_portuguese/pt_bosque-ud-train.conllu
2022-10-18 10:57:38,638 Dev: /home/davibarreira/.flair/datasets/ud_portuguese/pt_bosque-ud-dev.conllu
2022-10-18 10:57:38,639 Test: /home/davibarreira/.flair/datasets/ud_portuguese/pt_bosque-ud-test.conllu
Token[6]: "o" tensor([ 2.7388e-01,  1.1287e-01, -9.0481e-02, -1.1496e-01,  1.1164e-01,
         3.9364e-02, -5.3077e-02,  1.2918e-01, -3.0792e-02, -1.0750e-01,
        -1.6031e-02, -2.5077e-02, -7.7410e-02,  8.9467e-02, -2.1526e-01,
        -1.2137e-01,  8.9173e-02, -1.8762e-01, -1.2617e-02, -9.3670e-02,
        -1.2754e-01, -3.3130e-01,  3.6463e-02, -4.7890e-02,  4.7475e-02,
        -1.6050e-01, -2.0617e-01,  1.3285e-02,  3.3718e-03,  2.1074e-02,
         3.9722e-01, -2.9204e-02, -1.0872e-01, -8.7509e-03, -1.8711e-01,
        -1.5177e-01,  3.8992e-02,  3.0053e-02, -7.5466e-02,  1.8972e-01,

Considere agora a palavra "bom". Vamos criar o seu vetor. A pergunta é, qual a palavra no nosso vocabulário
que é o oposto de "bom"? Para isso, obtemos o embedding de "bom" e em seguida o multiplicamos por -1. Por fim,
buscamos no dicionário a palavra que mais se aproxima desse embedding.

In [18]:
bom = Sentence('bom')
fasttext.embed(bom)

emb = -1*np.array(bom[0].embedding.tolist())

In [19]:
cosine_similarity([emb],[list(emb)])[0][0]

1.0000000000000007

In [20]:
# `emb` é o embedding

def find_closest_matching_word(emb, vocab):
    max_match = -1
    for word in vocab:
        match = cosine_similarity([emb], [word.embedding.tolist()])[0][0]
        if match > max_match:
            max_match = match
            closest_matching_word = word.text
    return closest_matching_word

In [21]:
find_closest_matching_word(emb, vocab)

'Sade'

... O resultado não deu bem o que esperávamos. 

Isso quer dizer que nosso embedding não está performando tão bem quanto desejado.

## Flair Embeddings

Como mostramos no exemplo passado, o embedding original "FastText" não se mostrou muito preciso.
No "Flair" temos um embedding especial chamado de "Flair Embedding".
Esse embedding é bastante poderoso, e utiliza o contexto na hora de gerar o embedding. Ou seja,
ele não considera a palavra isoladamente. Isso é fundamental, pois sabemos que a mesma palavra
pode ter significados totalmente diferentes dado um contexto. Por exemplo, "manga" a fruta e "manga" de uma camisa.
Se usarmos um único embedding para representar a palavra "manga", não faz sentido falar da proximidade da palavra
"manga" e "caju", pois "manga" também é a parte de uma roupa, que nada tem a ver com uma fruta.

A primeira coisa a se entender dos Flair Embeddings é que existem dois tipos, o "forward" e o "backward". O "forward"
considera como contexto somente o que vem antes da token, enquanto que o "backward" considera
como contexto somente o que vem depois.
Assim, nas senteças "manga da camisa" e "manga boa de comer", o embedding "forward" da token "manga" será igual para as duas sentenças,
mas o "backward" será diferente.

In [22]:
from flair.embeddings import FlairEmbeddings

flair_embedding_forward = FlairEmbeddings('pt-forward')
flair_embedding_backward = FlairEmbeddings('pt-backward')

In [23]:
s1 = Sentence("manga da camisa")
s2 = Sentence("manga boa de comer")

flair_embedding_forward.embed(s1)
flair_embedding_forward.embed(s2)

print(s1[0].embedding.tolist() == s2[0].embedding.tolist())

flair_embedding_backward.embed(s1)
flair_embedding_backward.embed(s2)
print(s1[0].embedding.tolist() == s2[0].embedding.tolist())

True
False


In [24]:
s1 = Sentence('ele é uma pessoa boa')
s2 = Sentence('ele é uma pessoa excelente')
flair_embedding_forward.embed(s1)
flair_embedding_forward.embed(s2)

cosine_similarity([s1[-1].embedding.tolist()],[s2[-1].embedding.tolist()])

array([[0.86898918]])

Outro aspecto interessante do Flair é que ele é capaz de lidar com palavras fora do vocabulário. Veja, caso escrevamos
uma palavra errada, ele ainda é capaz de fazer um embedding, inclusive retornando uma similaridade boa.

In [25]:
s1 = Sentence('ele é uma pessoa boa')
s2 = Sentence('ele é uma pessoa boaa')
flair_embedding_forward.embed(s1)
flair_embedding_forward.embed(s2)

cosine_similarity([s1[-1].embedding.tolist()],[s2[-1].embedding.tolist()])

array([[0.76655116]])

## PooledFlairEmbeddings

Esse outro embedding mantém um contexto global das tokens, de forma que o seu embedding vai mudando a medida que mais
ocorrências da mesma palavra vão acontecendo. Assim, a mesma palavra com mesmo contexto poderá ter um novo embedding, caso
já tenha ocorrido na sentença em algum outro momento.

O uso desse embedding é "identico" ao FlairEmbedding.

In [27]:
from flair.embeddings import PooledFlairEmbeddings

pooled_forward  = PooledFlairEmbeddings('pt-forward')
pooled_backward = PooledFlairEmbeddings('pt-backward')

## Stacked embeddings

Mas, existem palavras que o contexto envolve tanto o que vem antes como o que vem depois... Portanto, precisamos
de alguma forma combinar os embeddings "forward" e "backward". Para isso, usamos o Stacked Embedding.

In [29]:
from flair.embeddings import StackedEmbeddings
fasttext = WordEmbeddings('pt')
flair_fw = FlairEmbeddings('pt-forward')
flair_bw = FlairEmbeddings('pt-backward')
combined_embeddings_list = [fasttext, flair_fw, flair_bw]

stack = StackedEmbeddings(combined_embeddings_list)

Novamente, esse embedding funciona da mesma forma que os demais. 

## Transformer Embedding

Além de todos esses, o Flair possui ainda mais outras opções de embedding. Uma especialmente útil são os embeddings usando Transformers.
Podemos tanto usar modelos pré-treinados do Hugging Face, como podemos usar modelos próprios, contanto que sua arquitetura tenha suporte no Flair.

In [33]:
from legalnlp.get_premodel import *
# get_premodel('bert')

In [40]:
from flair.embeddings import TransformerWordEmbeddings

In [47]:
sentenca = Sentence('O juiz julgou procedente.')
embeddings = TransformerWordEmbeddings('./models/BERTikal/', layers='-1', layer_mean=False)
embeddings.embed(sentenca)
print(sentenca[0].embedding.size())

torch.Size([768])


In [48]:
sentenca.clear_embeddings()

## Document Embedding

Mostramos como representar palavras como vetores. O mesmo pode ser feito para textos como um todo. Ou seja,
podemos representar um documento como um vetor, e usar a semelhança de cossenos para avaliar quão parecidos são dois documentos.
Esse tipo de ideia nos permite fazer coisas como clusterizar documentos.

In [51]:
from flair.embeddings import TransformerDocumentEmbeddings

embedding = TransformerDocumentEmbeddings('./models/BERTikal/')

sentenca = Sentence('O juiz julgou procedente.')
embedding.embed(sentenca)

print(sentenca.embedding.size(),sentenca.embedding)

torch.Size([768]) tensor([-3.6046e-02,  3.8304e-01, -5.0739e-01,  4.7760e-01,  1.0422e-01,
         9.2792e-01, -3.1415e-01, -2.3043e-01, -3.1315e-01,  5.7292e-01,
         1.5351e-01,  6.1514e-01,  6.3847e-02, -5.4561e-01,  5.1991e-01,
         3.6589e-01,  9.9137e-01, -6.0815e-02,  6.9953e-02,  3.7254e-01,
        -1.2381e+00,  9.2895e-01, -6.8616e-02, -6.5012e-02,  7.1273e-01,
         1.4171e-01,  1.3703e-01,  1.4975e-01,  3.4588e-01, -9.2155e-02,
         1.8884e-01,  3.2722e-01,  4.6638e-01,  8.0505e-02,  6.7849e-02,
         4.6737e-01,  4.4417e-01,  1.0906e+00,  1.1045e-01,  1.9594e-01,
         2.7253e-01,  3.1203e-01,  3.8897e-01, -7.7073e-01, -2.0878e-01,
        -3.4655e-01,  3.2428e-01,  5.9096e-01, -1.1603e-01,  7.6025e-02,
        -4.2636e-01,  2.1179e-01, -5.4417e-01, -3.7778e-01, -4.8404e-01,
        -2.2989e-01,  2.3265e-01, -2.9809e-01, -1.8232e-01, -3.1540e-01,
         5.6045e-01, -3.9891e-01, -3.9693e-02, -7.5340e-01, -1.2653e-01,
        -8.0830e-01, -1.5234e-01,